<a href="https://colab.research.google.com/github/siddhartha201/CSC-752-Project/blob/main/.ipynb_checkpoints/CNN%20model%20on%20Medical%20Imaging-checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives


In [ ]:
!git clone https://github.com/siddhartha201/CSC-752-Project.git


In [ ]:

import os
import shutil
import math
import pandas as pd

# Define paths
src_folder = 'BrEaST-Lesions_USG-images_and_masks'
train_folder = 'BrEaST-Lesions_USG-images_and_masks/train'
test_folder = 'BrEaST-Lesions_USG-images_and_masks/test'
excel_file_path = 'BrEaST-Lesions-USG-clinical-data-Dec-15-2023.xlsx'

# Read the Excel file
df = pd.read_excel(excel_file_path)

df['Classification'].unique()

In [ ]:
df.iloc[40:47]

In [ ]:

# Fill missing names in column 'C' based on their row index
for index in df.index:
    if pd.isna(df.at[index, 'Mask_tumor_filename']):  # Check if the value is NaN
        # Generate the name pattern based on index
        df.at[index, 'Mask_tumor_filename'] = f'case{index+1:03d}_tumor.png'

# Extract image names and their corresponding classes
image_names_b_train = df['Image_filename'][0:].tolist()  # Adjusting for zero-based index
image_names_c_train = df['Mask_tumor_filename'][0:].tolist()  # Adjusting for zero-based index
classes_train = df['Classification'][0:].replace('normal', 'benign').tolist()  # Treating 'normal' as 'benign'

# Create a list for train images with desired pairing
train_images = []

# Loop through the classes and corresponding images from both columns
for i in range(len(classes_train)):
    # Pair the first column's image name with the class
    train_images.append((image_names_b_train[i], classes_train[i]))
    # Pair the second column's image name with the class
    train_images.append((image_names_c_train[i], classes_train[i]))

# Create directories for classes in both train and test folders
os.makedirs(os.path.join(train_folder, 'malignant'), exist_ok=True)
os.makedirs(os.path.join(train_folder, 'benign'), exist_ok=True)
os.makedirs(os.path.join(test_folder, 'malignant'), exist_ok=True)
os.makedirs(os.path.join(test_folder, 'benign'), exist_ok=True)

# Shuffle and split into train and test datasets
num_train_images = math.ceil(len(train_images) * 0.9)  # 90% for training
train_images_split = train_images[:num_train_images]  # First 90% for training
test_images = train_images[num_train_images:]  # Remaining for testing

# Function to copy images to their respective class folders
def copy_images(image_list, folder):
    for img_name, label in image_list:
        src_file = os.path.join(src_folder, img_name)
        dst_folder = os.path.join(folder, label)
        dst_file = os.path.join(dst_folder, img_name)

        if os.path.isfile(src_file):  # Ensure the file exists before copying
            shutil.copy(src_file, dst_file)

# Copy images for training
copy_images(train_images_split, train_folder)

# Copy images for testing
copy_images(test_images, test_folder)

print(f"Copied {len(train_images_split)} training images to {train_folder}")
print(f"Copied {len(test_images)} testing images to {test_folder}")



In [ ]:
df.iloc[250:]

In [ ]:

import os

# Specify the directory path
train_folder = '/content/CSC-752-Project/BrEaST-Lesions_USG-images_and_masks/train'

# Count the number of files in the directory
file_count = sum(len(files) for _, _, files in os.walk(train_folder))

print(f"Number of files in '{train_folder}': {file_count}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# directory
base_dir = "BrEaST-Lesions_USG-images_and_masks"

# Training and testing
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# rescaling
train_datagen = ImageDataGenerator(rescale=1/255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1/255)

# Create a stream for training
train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary'
)

# Create a stream of images for testing
test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='binary'
)


In [ ]:
# Initialize the model
model = Sequential()

# Add regularization to the convolutional layers and include dropout layers
model.add(Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(256,256,3), kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(3, 3))

model.add(Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(3, 3))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D(3, 3))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))  # Add dropout after the dense layer

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy', AUC(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()])

# Model summary
model.summary()

# Add EarlyStopping when the validation loss starts increasing
early_stopping = EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // train_dataset.batch_size,
    epochs=100,
    validation_data=test_dataset,
    validation_steps=test_dataset.samples // test_dataset.batch_size,
    callbacks=[early_stopping])


In [ ]:
# Print final accuracy and loss
final_train_accuracy = history.history['accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"Final Training Accuracy: {final_train_accuracy}")
print(f"Final Training Loss: {final_train_loss}")
print(f"Final Validation Accuracy: {final_val_accuracy}")
print(f"Final Validation Loss: {final_val_loss}")


In [ ]:
import matplotlib.pyplot as plt

# Plot for accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot for loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('CNN Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plot for accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [12]:
#model.save('C:/Users/alan/Medical Image Project/CNN model.h5')